In [4]:
from socket import *
import json
import base64

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database=""
)

serverPort = 1201
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(('', serverPort))
serverSocket.listen(1)
connectionSocket, addr = serverSocket.accept()

print('The server is ready to receive')
def getRequest():
    while True:
        #cria o socket pro cliente e estabelece a conexão
        
        BUFF_SIZE = 4096 # 4 KiB
        data = b''
        while True:
            part = connectionSocket.recv(BUFF_SIZE)
            data += part
            if len(part) < BUFF_SIZE:
                # either 0 or end of data
                break
        #sentence = connectionSocket.recv(8096)
    
    dado = json.loads(data.decode())
    return dado
def sendReply(resp_json):
    connectionSocket.send(resp_json)
def cadastrar(json_data):
    fileData = base64.b64decode(json_data['conteudo'])
    # make file
    nome = "livros/"+json_data['nome']+".pdf"
    newFile = open(nome, "wb")
    # write to file
    newFile.write(fileData)
    
    mycursor = mydb.cursor()

    sql = "INSERT INTO livraria (nome, autor, conteudo) VALUES (%s, %s, %s)"
    val = (json_data['nome'],json_data['autor'],json_data['conteudo'])
    mycursor.execute(sql, val)

    mydb.commit()
    
    x = {'error':'false'}
    sendJson = json.dumps(x)
    sendReply(sendJson)
def enviar(nome):
    mycursor = mydb.cursor()

    sql = "SELECT * FROM livraria WHERE nome = %s"
    val = (nome,)

    mycursor.execute(sql,val)

    myresult = mycursor.fetchall()
    return myresult
json_resp = getRequest()
if(json_resp['methodId'] == "1"):
    cadastrar(json_resp)
else:
    dado = enviar(json_resp['nome'])
    x = {'nome':dado[0],'autor':dado[1],'conteudo':dado[2]}
    json_send = json.dumps(x)
    sendReply(json_send)
connectionSocket.close()

InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (111 Connection refused)